# Analyse in Fiji and plot using R

This notebook demonstrates how to analyze a batch of images associated to the paper ['Subdiffraction imaging of centrosomes reveals higher-order organizational features of pericentriolar material.'](http://dx.doi.org/10.1038/ncb2591) using the scripting facility available in [Fiji](https://fiji.sc/).
In the paper, Images were analyzed using Matlab scripts. In this notebook, we use the *Auto Threshold* and *Analyze Particles* Fiji plugins. The results are saved in a CVS file. We then use R to plot the results and reproduce graphics from the paper.

## Analyse data in Fiji using the scripting facility

### Set up

1. Install Fiji
2. Install the OMERO.insight-ij plugin. See http://help.openmicroscopy.org/imagej.html#plugins
3. Installing the plugin adds the dependencies required to connect to IDR using the *Script Editor* of Fiji.

### Analysis
Analysing the full set of images will take roughly 30mins depending on the network speed.
To analyse the data, follow the steps below:

1. Select the Project https://idr.openmicroscopy.org/webclient/?show=project-51
2. Note its ID displayed on the right-hand panel
3. Launch Fiji
4. Go to *File > New > Script...*
5. A dialog pops up. In the *Language* menu select *Python*
6. Copy the content of the script [idr0021.jy](includes/idr0021.jy) into the text editor
7. Edit USERNAME and PASSWORD
8. Click *Run*
9. The script will process all Images in the specified Project. The following steps will be applied to each image:
    - Convert floating-point pixel-type to 8-bit using *8-bit* 
    - Apply *Auto Threshold* with the *method* set to *MaxEntropy* and *stack* option selected
    - Apply *Analyze Particles* with the *Size* set to *10-Infinity* and the *Pixel units* checked.
    - Parse the results to only keep a summary of the statistics of the channel which name is matching the name of the dataset are kept. To identify the channel, we read the map annotation associated to the image.The map annotation contains pairs (emission wavelength, protein).
    - Save the summary as a CSV file. One file per Image
10. All the CSV files are then aggregated into a single CSV file.

## Plot with R

A simple example which reads the CSV file, performs some basic statistics on it and creates a plot which should be similar to [Figure 1](https://www.nature.com/articles/ncb2591/figures/1).

## Load the data from the CSV file as R dataframe

In [ ]:
# Read the CSV file
home <- path.expand('~')
path <- paste0(home, "/notebooks/includes/idr0021_merged_results.csv")
df <- read.csv(path)

## Prepare the data

In [ ]:
# CEP120 is represented by two datasets'CEP120/20111106' and 'CEP120/20111209',
# just combine them to 'CEP120'
df$Dataset <- as.character(df$Dataset)
df[ df == 'CEP120/20111106' ] <- 'CEP120'
df[ df == 'CEP120/20111209' ] <- 'CEP120'
df$Dataset <- as.factor(df$Dataset)

# Remove the extreme (very likely wrong) values of the NEDD1-C1 dataset
df <- df[ which(df$Bounding_Box < 100), ]
# Use the bounding box of the biggest shapes for the specified channel
df$diameter <- sqrt(df$Bounding_Box)

## Plot the data

In [ ]:
# Order the data from lowest to highest mean diameter (to match the order of figure 1)
ag <-aggregate(df$diameter ~ df$Dataset, df, mean)
orderedDatasets <- factor(df$Dataset, levels=ag[order(ag$`df$diameter`), 'df$Dataset'])

plot(df$diameter ~ orderedDatasets, ylab='Bounding Box', xlab="Protein", cex.axis=0.5)

## Is there a significant difference between the proteins?

In [ ]:
# One-way analysis of variance:
fit <- aov(df$diameter ~ df$Dataset)
summary(fit)

# Two-sample Wilcoxon test ('Mann-Whitney') of all pairwise combinations:
combins <- combn(levels(df$Dataset), 2)
params_list <- split(as.vector(combins), rep(1:ncol(combins), each = nrow(combins)))
testResults <- data.frame()
for (param in params_list) {
  testdf <- subset(df, df$Dataset %in% param)
  pval <- wilcox.test(formula = diameter ~ Dataset, data = testdf)$p.value
  testResults<-rbind(testResults, data.frame(Protein_1=param[1], Protein_2=param[2], p_value=pval))
}
testResults

### License

Copyright (C) 2018 University of Dundee. All Rights Reserved.

This program is free software; you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation; either version 2 of the License, or (at your option) any later version.

This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details. You should have received a copy of the GNU General Public License along with this program; if not, write to the Free Software Foundation, Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.